In [37]:
import pandas as pd
import numpy as np
import math
from numba import njit

In [38]:
df = pd.read_csv('data.csv', delimiter=',')
# df = df.head(20)

df

,id,region,municipality,settlement,type,latitude_dd,longitude_dd
0,0,Республика Алтай,Шебалинский район,Каспа,с,5111,8601
1,1,Алтайский край,Смоленский,Молочный,п,5241,8497
2,2,Красноярский край,Казачинский район,Отношка,с,5738,9270
3,3,Республика Тыва,Каа-Хемский кожуун,Кундустуг,с,5157,9518
4,4,Красноярский край,Курагинский район,Щетинкино,с,5453,9344
...,...,...,...,...,...,...,...
6132,6132,Новосибирская область,Сузунский район,Мереть,с,5357,8239
6133,6133,Новосибирская область,Сузунский район,Маюрово,с,5431,8242
6134,6134,Алтайский край,Быстроистокский,Смоленский,п,5221,8466
6135,6135,Новосибирская область,Искитимский район,Малиновка,д,5466,8386


In [145]:
@njit(fastmath = True)
def create_mtrx(k, arr):
    dist_arr = np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            if i != j:
                dist_arr[i][j] = math.sqrt((arr[j][0] - arr[i][0])**2 + (arr[j][1] - arr[i][1])**2)

    return dist_arr

# def calculate_distance(x1, y1, x2, y2):
#     distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
#     return distance

@njit(fastmath = True)
def greedy_tsp(k, d_arr):
    start_idx = 3753
    visited = np.zeros(k, dtype=np.int64)
    visited[start_idx] = 1
    current_city = start_idx
    way = np.zeros(k + 1, dtype=np.int64)
    way[0] = way[k] = start_idx

    for i in range(1, k):
        min_distance = np.inf
        nearest_city = -1

        for city in range(k):
            if not visited[city] and d_arr[current_city][city] < min_distance:
                min_distance = d_arr[current_city][city]
                nearest_city = city
        way[i] = nearest_city
        visited[nearest_city] = 1
        current_city = nearest_city
    return way

@njit(fastmath = True)
def count_len(matrix, way):
    total_len = 0
    for i in range(len(way) - 1):
        total_len += matrix[way[i]][way[i + 1]]
    return total_len / 100

def save_csv(matrix, way):
    data = {'Id': list(way[:-1]), 'Predicted': []}
    for i in range(len(way) - 1):
        data["Predicted"].append(matrix[way[i]][way[i + 1]])
    df = pd.DataFrame(data)
    df.to_csv("res.csv", index=False, sep=",", float_format="%.3f")

In [133]:
num = df.shape[0]
coord_cols = np.array(df.loc[:, ['latitude_dd', 'longitude_dd']])
matrix = create_mtrx(num, coord_cols)
way = greedy_tsp(num, matrix)
count_len(matrix, way)

1074.3788465925031

In [146]:
save_csv(matrix, way)